In [1]:
import pandas as pd

def inspect_columns(csv_path):
    """
    Load a CSV file and print out its column names and a preview of the first few rows.
    """
    # Load the CSV
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    
    # Print column names
    print("Columns in the dataset:")
    for col in df.columns:
        print(f" - {col}")
    
    # Print a quick preview
    print("\nPreview of first 5 rows:")
    print(df.head())

if __name__ == "__main__":
    # Update this path if needed
    csv_path = r"C:\Users\c-x-h\OneDrive\Masaüstü\No_emoji_despa\data\despa_kJQP7kiw5Fk_comments\despa_kJQP7kiw5Fk_comments.csv"
    inspect_columns(csv_path)


Columns in the dataset:
 - comment_id
 - original_comment
 - comment_no_emojis
 - emojis_in_comment
 - emoji_text_representation
 - detected_language
 - author_name
 - published_at
 - published_at_unix
 - like_count

Preview of first 5 rows:
                   comment_id  \
0  Ugy0Kpypj3U4drELURJ4AaABAg   
1  UgyoE3fdtw3PKUElVhZ4AaABAg   
2  UgzQ7ZTX6Z7qVSD9C-J4AaABAg   
3  Ugz8x2qV2P8CqBoSzEJ4AaABAg   
4  UgzFbBsDyBIV6rYe0AJ4AaABAg   

                                    original_comment  \
0                      Anyone here 2025 may Thursday   
1  <a href="https://www.youtube.com/watch?v=tOHIT...   
2                   5-may-2025 who’s still listening   
3                                   2025 now enjoy 🤍   
4                             Cristiano Ronaldo 2017   

                                   comment_no_emojis emojis_in_comment  \
0                      Anyone here 2025 may Thursday               NaN   
1  <a href="https://www.youtube.com/watch?v=tOHIT...               NaN   


In [3]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install kneed

Note: you may need to restart the kernel to use updated packages.


In [7]:
#!/usr/bin/env python3
"""
Full pipeline for filtering and selecting “visit”-related comments in English,
using an existing 'comment_no_emojis' column (no emoji stripping needed).

Requires:
    pip install pandas beautifulsoup4 numpy scikit-learn matplotlib kneed sentence-transformers
"""

import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from kneed import KneeLocator

def preprocess_text(text: str) -> str:
    """
    Remove URLs, HTML tags, and extra whitespace from text.
    """
    # remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)
    # remove HTML tags using built-in parser
    text = BeautifulSoup(text or "", "html.parser").get_text()
    # collapse whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def main():
    # Paths
    input_path  = r"C:\Users\c-x-h\OneDrive\Masaüstü\No_emoji_despa\data\despa_kJQP7kiw5Fk_comments\despa_kJQP7kiw5Fk_comments.csv"
    output_path = r"C:\Users\c-x-h\OneDrive\Masaüstü\filtered_visit_comments.csv"
    plot_path   = r"C:\Users\c-x-h\OneDrive\Masaüstü\similarity_distribution.png"

    # 1. Load data
    df = pd.read_csv(input_path, encoding='utf-8', low_memory=False)
    print("Columns in dataset:", df.columns.tolist())

    # 2. Filter by language
    if 'detected_language' not in df.columns:
        raise KeyError("Column 'detected_language' not found.")
    df = df[df['detected_language'] == 'en'].copy()
    print(f"Rows after language filter: {len(df)}")

    # 3. Preprocess text using comment_no_emojis
    if 'comment_no_emojis' not in df.columns:
        raise KeyError("Column 'comment_no_emojis' not found.")
    df['clean_text'] = df['comment_no_emojis'].astype(str).apply(preprocess_text)

    # 4. Regex-based keyword filtering
    pattern = re.compile(
        r'\b(visit|travel|trip|go|place|wish I could|want to visit|Puerto Rico|Porto Rico|Porto Riko)\b',
        flags=re.IGNORECASE
    )
    df_filtered = df[df['clean_text'].str.contains(pattern, na=False)].copy()
    print(f"Rows after keyword filter: {len(df_filtered)}")

    # 5. Compute embeddings
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    comments = df_filtered['clean_text'].tolist()
    embeddings = model.encode(comments, show_progress_bar=True)

    # 6. Seed sentence set & centroid
    seed_sentences = [
        "I want to visit this place",
        "I'd love to visit here",
        "This looks like a place I should go"
    ]
    seed_embeddings = model.encode(seed_sentences)
    centroid = np.mean(seed_embeddings, axis=0, keepdims=True)

    # 7. Cosine similarity
    sims = cosine_similarity(embeddings, centroid).flatten()

    # 8. Dynamic threshold via knee detection
    sims_sorted = np.sort(sims)[::-1]
    knee_locator = KneeLocator(
        range(len(sims_sorted)), sims_sorted,
        curve='convex', direction='decreasing'
    )
    knee = knee_locator.knee
    threshold = sims_sorted[knee] if knee is not None else 0.7
    print(f"Detected similarity threshold: {threshold:.3f}")

    # 9. Save similarity distribution plot
    plt.figure(figsize=(8, 4))
    plt.plot(sims_sorted, label='Sorted Similarities')
    if knee is not None:
        plt.axvline(knee, color='red', linestyle='--', label=f'Knee at idx={knee}')
    plt.title("Cosine Similarity Distribution")
    plt.xlabel("Ranked Comments")
    plt.ylabel("Similarity")
    plt.legend()
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()
    print(f"Similarity distribution saved to: {plot_path}")

    # 10. Select high-scoring comments and save
    selected_idx = np.where(sims >= threshold)[0]
    df_selected = df_filtered.iloc[selected_idx]
    print(f"Number of comments selected: {len(df_selected)}")
    df_selected.to_csv(output_path, index=False, encoding='utf-8')
    print(f"Filtered comments saved to: {output_path}")

if __name__ == "__main__":
    main()


Columns in dataset: ['comment_id', 'original_comment', 'comment_no_emojis', 'emojis_in_comment', 'emoji_text_representation', 'detected_language', 'author_name', 'published_at', 'published_at_unix', 'like_count']
Rows after language filter: 1094704


C:\Users\c-x-h\AppData\Local\Temp\ipykernel_8468\2383492124.py:57: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered = df[df['clean_text'].str.contains(pattern, na=False)].copy()


Rows after keyword filter: 16136


d:\a\envs\bertopic_env\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\c-x-h\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falli

Detected similarity threshold: 0.703
Similarity distribution saved to: C:\Users\c-x-h\OneDrive\Masaüstü\similarity_distribution.png
Number of comments selected: 22
Filtered comments saved to: C:\Users\c-x-h\OneDrive\Masaüstü\filtered_visit_comments.csv


In [14]:
pip install faiss 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [2]:
#!/usr/bin/env python3
"""
Extended pipeline to increase recall of “visit”-related comments without FAISS:
1. Broad regex keyword filtering
2. Expanded seed set with per-seed max similarity
3. Dynamic + manual threshold
4. Clustering to capture thematic groups
5. Top-N similarity retrieval instead of FAISS

Requirements:
    pip install pandas beautifulsoup4 numpy scikit-learn matplotlib sentence-transformers hdbscan kneed
"""

import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from kneed import KneeLocator
import matplotlib.pyplot as plt

def preprocess_text(text: str) -> str:
    """
    Remove URLs, HTML tags, and extra whitespace.
    """
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = BeautifulSoup(text or "", "html.parser").get_text()
    return re.sub(r'\s+', ' ', text).strip()

def main():
    # Paths
    input_csv   = r"C:\Users\c-x-h\OneDrive\Masaüstü\No_emoji_despa\data\despa_kJQP7kiw5Fk_comments\despa_kJQP7kiw5Fk_comments.csv"
    output_csv  = r"C:\Users\c-x-h\OneDrive\Masaüstü\extended_filtered_comments.csv"
    plot_path   = r"C:\Users\c-x-h\OneDrive\Masaüstü\similarity_distribution_extended.png"

    # 1. Load data
    df = pd.read_csv(input_csv, encoding='utf-8', low_memory=False)
    print("Columns loaded:", df.columns.tolist())

    # 2. Preprocess text
    df['clean_text'] = df['comment_no_emojis'].astype(str).apply(preprocess_text)

    # 3. Broad regex filtering
    keywords = [
        r'visit', r'travel', r'trip', r'go', r'place',
        r'wish I could', r'want to visit', r'Puerto Rico',
        r'Porto Rico', r'Porto Riko', r'vacation',
        r'destination', r'sightseeing', r'getaway',
        r'explore', r'backpack', r'itinerary'
    ]
    pattern = re.compile(r'\b(' + '|'.join(keywords) + r')\b', flags=re.IGNORECASE)
    df_kw = df[df['clean_text'].str.contains(pattern, na=False)].copy()
    print(f"After broad regex filter: {len(df_kw)} comments")

    # 4. Compute embeddings
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    texts = df_kw['clean_text'].tolist()
    embeddings = model.encode(texts, show_progress_bar=True)
    embeddings = np.array(embeddings, dtype=np.float32)

    # 5. Seed sentences and embeddings
    seed_sentences = [
        "I want to visit this place",
        "I'd love to visit here",
        "This looks like a place I should go",
        "I've always wanted to go here",
        "This destination is on my bucket list",
        "Put this place on my must-visit list",
        "Vacationing in this area"
    ]
    seed_emb = model.encode(seed_sentences)
    seed_emb = np.array(seed_emb, dtype=np.float32)

    # 6. Per-seed similarity and maximum per comment
    sims_per_seed = cosine_similarity(embeddings, seed_emb)  # shape: (n_comments, n_seeds)
    max_sims = sims_per_seed.max(axis=1)

    # 7. Determine threshold: dynamic knee + manual lower bound
    sims_sorted = np.sort(max_sims)[::-1]
    knee_idx = KneeLocator(range(len(sims_sorted)), sims_sorted,
                           curve='convex', direction='decreasing').knee
    dynamic_thresh = sims_sorted[knee_idx] if knee_idx is not None else 0.6
    manual_thresh = 0.6
    threshold = min(dynamic_thresh, manual_thresh)
    print(f"Threshold chosen: {threshold:.3f} (dynamic: {dynamic_thresh:.3f}, manual: {manual_thresh})")

    # Plot similarity distribution
    plt.figure(figsize=(8,4))
    plt.plot(sims_sorted, label='Max per-seed similarities')
    if knee_idx is not None:
        plt.axvline(knee_idx, color='red', linestyle='--', label=f'Knee idx={knee_idx}')
    plt.title("Extended Cosine Similarity Distribution")
    plt.xlabel("Ranked Comments")
    plt.ylabel("Similarity")
    plt.legend()
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()
    print(f"Similarity distribution plot saved to: {plot_path}")

    # 8. Filter by threshold
    idx_thresh = set(np.where(max_sims >= threshold)[0])
    print(f"Candidates after threshold filter: {len(idx_thresh)}")

    # 9. Clustering with KMeans to pick semantically similar clusters
    n_clusters = 10
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(embeddings)
    centers = kmeans.cluster_centers_
    avg_seed = seed_emb.mean(axis=0, keepdims=True)
    cluster_sims = cosine_similarity(centers, avg_seed).flatten()
    good_clusters = {i for i, sim in enumerate(cluster_sims) if sim >= threshold}
    idx_clusters = {i for i, lbl in enumerate(kmeans.labels_) if lbl in good_clusters}
    print(f"Candidates from clusters: {len(idx_clusters)}")

    # 10. Top-N similarity retrieval to boost recall
    top_n = 5000
    top_indices = np.argsort(max_sims)[::-1][:top_n]
    idx_topn = set(top_indices)
    print(f"Candidates from Top-N ({top_n}): {len(idx_topn)}")

    # 11. Combine all candidate indices
    all_idx = idx_thresh.union(idx_clusters).union(idx_topn)
    df_selected = df_kw.iloc[list(all_idx)].reset_index(drop=True)
    print(f"Total unique candidates: {len(df_selected)}")

    # 12. Save results
    df_selected.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"Extended filtered comments saved to: {output_csv}")

if __name__ == "__main__":
    main()


Columns loaded: ['comment_id', 'original_comment', 'comment_no_emojis', 'emojis_in_comment', 'emoji_text_representation', 'detected_language', 'author_name', 'published_at', 'published_at_unix', 'like_count']


C:\Users\c-x-h\AppData\Local\Temp\ipykernel_15392\166916041.py:54: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_kw = df[df['clean_text'].str.contains(pattern, na=False)].copy()


After broad regex filter: 27893 comments


Batches: 100%|██████████| 872/872 [07:23<00:00,  1.96it/s]


Threshold chosen: 0.600 (dynamic: 0.637, manual: 0.6)
Similarity distribution plot saved to: C:\Users\c-x-h\OneDrive\Masaüstü\similarity_distribution_extended.png
Candidates after threshold filter: 173
Candidates from clusters: 5474
Candidates from Top-N (5000): 5000
Total unique candidates: 8585
Extended filtered comments saved to: C:\Users\c-x-h\OneDrive\Masaüstü\extended_filtered_comments.csv


In [1]:
import pandas as pd

# Replace backslashes with raw-string or double backslashes
file_path = r"C:\Users\c-x-h\OneDrive\Masaüstü\No_emoji_despa\data\despa_kJQP7kiw5Fk_comments\despa_kJQP7kiw5Fk_comments.csv"

# Read only the header row to get columns (faster for large files)
df = pd.read_csv(file_path, nrows=0)

# Print the list of column names
print(df.columns.tolist())


['comment_id', 'original_comment', 'comment_no_emojis', 'emojis_in_comment', 'emoji_text_representation', 'detected_language', 'author_name', 'published_at', 'published_at_unix', 'like_count']


In [2]:
import os
import pandas as pd

# 1. Load the full dataset
file_path = r"C:\Users\c-x-h\OneDrive\Masaüstü\No_emoji_despa\data\despa_kJQP7kiw5Fk_comments\despa_kJQP7kiw5Fk_comments.csv"
df = pd.read_csv(file_path)

# 2. Filter to English-language comments
df_en = df[df['detected_language'] == 'en'].copy()

# 3. Parse 'published_at' and extract year
df_en['published_at'] = pd.to_datetime(df_en['published_at'])
df_en['year'] = df_en['published_at'].dt.year

# 4. For each year, save the two columns into its own folder
for year in sorted(df_en['year'].unique()):
    out_dir = f"search_{year}"
    os.makedirs(out_dir, exist_ok=True)
    
    # Select only the two columns
    df_year = df_en.loc[df_en['year'] == year, ['comment_no_emojis', 'published_at']]
    
    # Write to CSV
    out_path = os.path.join(out_dir, f"comments_{year}.csv")
    df_year.to_csv(out_path, index=False)
    
    print(f"Saved {len(df_year)} rows to {out_path}")


Saved 304697 rows to search_2017\comments_2017.csv
Saved 187660 rows to search_2018\comments_2018.csv
Saved 157922 rows to search_2019\comments_2019.csv
Saved 215489 rows to search_2020\comments_2020.csv
Saved 118214 rows to search_2021\comments_2021.csv
Saved 49052 rows to search_2022\comments_2022.csv
Saved 26821 rows to search_2023\comments_2023.csv
Saved 26282 rows to search_2024\comments_2024.csv
Saved 8567 rows to search_2025\comments_2025.csv


In [3]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer, util

# ─── Configuration ─────────────────────────────────────────────────────────────
# Path to your original CSV
FILE_PATH = r"C:\Users\c-x-h\OneDrive\Masaüstü\No_emoji_despa\data\despa_kJQP7kiw5Fk_comments\despa_kJQP7kiw5Fk_comments.csv"

# N-gram filtering keywords
KEYWORDS = [
    "visit", "travel", "trip", "go", "place",
    "wish i could", "want to visit",
    "puerto rico", "porto rico", "porto riko"
]

# Sentence-Transformer model and seed sentence
MODEL_NAME    = "all-MiniLM-L6-v2"
SEED_SENTENCE = "I want to visit this place."

# Semantic similarity parameters
TOP_K      = 100
THRESHOLD  = 0.6

# ─── Load & Preprocess ─────────────────────────────────────────────────────────
df = pd.read_csv(FILE_PATH)

# 1. Keep only English comments
df = df[df["detected_language"] == "en"].copy()

# 2. Parse dates & extract year
df["published_at"] = pd.to_datetime(df["published_at"])
df["year"] = df["published_at"].dt.year

# ─── Set up N-gram Vectorizer ──────────────────────────────────────────────────
vectorizer = CountVectorizer(
    ngram_range=(1, 3),
    vocabulary=KEYWORDS,
    lowercase=True,
    token_pattern=r"(?u)\b\w+\b"
)

# ─── Set up Semantic Model ─────────────────────────────────────────────────────
model   = SentenceTransformer(MODEL_NAME)
seed_emb = model.encode(SEED_SENTENCE, convert_to_tensor=True)

# ─── Process Year by Year ──────────────────────────────────────────────────────
for year, group in df.groupby("year"):
    out_dir = f"search_{year}"
    os.makedirs(out_dir, exist_ok=True)

    # Save original comments for reference
    comments_path = os.path.join(out_dir, f"comments_{year}.csv")
    group[["comment_no_emojis", "published_at"]].to_csv(comments_path, index=False)

    # 1) Word- and Phrase-Based Filtering
    X      = vectorizer.transform(group["comment_no_emojis"])
    mask   = (X.sum(axis=1).A1 > 0)
    word_df = group.loc[mask, ["comment_no_emojis", "published_at"]].copy()

    word_path = os.path.join(out_dir, f"word_filtered_{year}.csv")
    word_df.to_csv(word_path, index=False)

    # 2) Semantic Similarity Scoring
    #    Embed the filtered comments (in batches)
    comments = word_df["comment_no_emojis"].tolist()
    embeddings = model.encode(
        comments,
        convert_to_tensor=True,
        batch_size=64,
        show_progress_bar=True
    )

    #    Compute cosine similarity against the seed
    sims = util.cos_sim(embeddings, seed_emb).squeeze(1)
    word_df["similarity_score"] = sims.cpu().numpy()

    #    Sort by descending similarity
    sorted_df = word_df.sort_values("similarity_score", ascending=False)

    # 2a) Top-K selection
    topk_df = sorted_df.head(TOP_K)
    topk_path = os.path.join(out_dir, f"semantic_topk_{year}.csv")
    topk_df.to_csv(topk_path, index=False)

    # 2b) Threshold filtering
    thresh_df = sorted_df[sorted_df["similarity_score"] >= THRESHOLD]
    thresh_path = os.path.join(out_dir, f"semantic_threshold_{year}.csv")
    thresh_df.to_csv(thresh_path, index=False)

    print(
        f"Year {year}: "
        f"{len(group)} total → "
        f"{len(word_df)} word-filtered → "
        f"{len(topk_df)} top-{TOP_K} → "
        f"{len(thresh_df)} ≥ {THRESHOLD}"
    )


d:\a\envs\bertopic_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 101/101 [01:17<00:00,  1.31it/s]


Year 2017: 304697 total → 6439 word-filtered → 100 top-100 → 10 ≥ 0.6


Batches: 100%|██████████| 35/35 [00:26<00:00,  1.30it/s]


Year 2018: 187660 total → 2178 word-filtered → 100 top-100 → 8 ≥ 0.6


Batches: 100%|██████████| 33/33 [00:23<00:00,  1.43it/s]


...

Year 2023: 26821 total → 288 word-filtered → 100 top-100 → 0 ≥ 0.6


Batches: 100%|██████████| 4/4 [00:04<00:00,  1.15s/it]


Year 2024: 26282 total → 245 word-filtered → 100 top-100 → 1 ≥ 0.6


Batches: 100%|██████████| 2/2 [00:02<00:00,  1.47s/it]

Year 2025: 8567 total → 107 word-filtered → 100 top-100 → 0 ≥ 0.6


In [5]:
import pandas as pd
import glob
import os

# ─── Adjust this if your folders are elsewhere ─────────────────────────────────
BASE_PATTERN = "Travel_Search/search_*/semantic_topk_*.csv"

# ─── Read & combine ─────────────────────────────────────────────────────────────
all_dfs = []
for path in glob.glob(BASE_PATTERN):
    df = pd.read_csv(path)
    # Convert published_at to just the year (int or str)
    df["published_at"] = pd.to_datetime(df["published_at"]).dt.year
    all_dfs.append(df)

if not all_dfs:
    raise FileNotFoundError(f"No files matched pattern {BASE_PATTERN}")

combined = pd.concat(all_dfs, ignore_index=True)

# ─── Write to Excel ─────────────────────────────────────────────────────────────
output_path = "combined_semantic_topk.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    combined.to_excel(writer, index=False, sheet_name="TopK")

print(f"Wrote {len(combined)} rows into {output_path}")


Wrote 900 rows into combined_semantic_topk.xlsx


In [8]:
import glob
import pandas as pd
import os

# ─── Configuration ─────────────────────────────────────────────────────────────
# Pattern to find your per-year semantic_topk files
PATTERN = os.path.join("Travel_Search","search_*", "semantic_topk_*.csv")
OUTPUT_FILE = "topk_200_500.xlsx"

# ─── Find files ────────────────────────────────────────────────────────────────
paths = glob.glob(PATTERN)
if not paths:
    raise FileNotFoundError(f"No files matched pattern: {PATTERN}\n"
                            "Make sure you're running this script from the folder containing your search_<year> directories.")

print(f"Found {len(paths)} files:")
for p in paths:
    print("  ", p)

# ─── Read & normalize ─────────────────────────────────────────────────────────
dfs = []
for p in paths:
    df = pd.read_csv(p)
    # reduce published_at to just the year
    df["published_at"] = pd.to_datetime(df["published_at"]).dt.year
    dfs.append(df[["comment_no_emojis", "published_at", "similarity_score"]])

# ─── Combine ─────────────────────────────────────────────────────────────────
combined = pd.concat(dfs, ignore_index=True)
combined = combined.sort_values("similarity_score", ascending=False)

# ─── Write Top-200 & Top-500 ───────────────────────────────────────────────────
with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
    combined.head(200).to_excel(writer, sheet_name="Top_200", index=False)
    combined.head(500).to_excel(writer, sheet_name="Top_500", index=False)

print(f"\nWrote Top 200 and Top 500 to '{OUTPUT_FILE}'")


Found 9 files:
   Travel_Search\search_2017\semantic_topk_2017.csv
   Travel_Search\search_2018\semantic_topk_2018.csv
   Travel_Search\search_2019\semantic_topk_2019.csv
   Travel_Search\search_2020\semantic_topk_2020.csv
   Travel_Search\search_2021\semantic_topk_2021.csv
   Travel_Search\search_2022\semantic_topk_2022.csv
   Travel_Search\search_2023\semantic_topk_2023.csv
   Travel_Search\search_2024\semantic_topk_2024.csv
   Travel_Search\search_2025\semantic_topk_2025.csv

Wrote Top 200 and Top 500 to 'topk_200_500.xlsx'
